In [98]:
import torch

t= torch.randn(10,120,1)

In [111]:
seq = [1,2,3,4,5]

a,*b, c = seq

a,b,c

(1, [2, 3, 4], 5)

In [112]:
# [0, *seq] # similar to l=[0]; l.extend(seq); l

In [5]:
import torch

words = ['worlds', 'beautiful', 'world', '<PAD>']

def encode_8bit(words):
    max_l = 0
    ts_list = []
    for w in words:
        ts_list.append(torch.ByteTensor(list(bytes(w, 'utf8'))))
        max_l = max(ts_list[-1].size()[0], max_l)

    w_t = torch.zeros((len(ts_list), max_l), dtype=torch.uint8)
    for i, ts in enumerate(ts_list):
        w_t[i, 0:ts.size()[0]] = ts
    return w_t

encode_8bit(words)

tensor([[119, 111, 114, 108, 100, 115,   0,   0,   0],
        [ 98, 101,  97, 117, 116, 105, 102, 117, 108],
        [119, 111, 114, 108, 100,   0,   0,   0,   0],
        [ 60,  80,  65,  68,  62,   0,   0,   0,   0]], dtype=torch.uint8)

In [30]:
from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem import WordNetLemmatizer
stop_words = set(stopwords.words('english')) 

def clean_and_encode(text, lemmatize=False , clean_punc=False, clean_stopwords=False):
    
    text = str(text) # in case of non-string values
    text = text.lower()
    text = TweetTokenizer().tokenize(text) # => list of tokens
    
    if lemmatize:
        text = [WordNetLemmatizer().lemmatize(token) for token in text]

    if clean_punc:
        text = [token for token in text if token not in punctuation]
    if clean_stopwords:
        text = [token for token in text if token not in stop_words]
    
    return encode_8bit(text)

In [32]:
testtext = "runin for senitur! #YOLO!"
clean_and_encode(testtext).shape

torch.Size([6, 7])

In [49]:
from torchtext.legacy.data import Field, BucketIterator, TabularDataset

def get_datasets(device, batch_size=50):

    # "fields" that process the columns in CSV files
    Tokens = Field(tokenize = lambda x:TweetTokenizer().tokenize(x), lower=True, batch_first=True)
#     Labels = torchtext.data.Field(batch_first=True)
    fields = [('text', Tokens), ('parent_text', Tokens)]
    
    #Process from CSV files
    train,test = TabularDataset.splits(
            path='data', train='train.csv', test='test.csv',
            format='csv', fields=fields, skip_header=True, 
            csv_reader_params = {'delimiter':'\t','quotechar':'、'})

    # build vocabularies based on what CSV files contained and create word2id mapping
    Tokens.build_vocab(train)

    #Batch iterator
    train_batches,test_batches = BucketIterator.splits(
            (train,test), batch_size=batch_size, shuffle=True, device=device,
             sort_within_batch=True, sort_key=lambda x: len(x.text))
    
    return train_batches, test_batches, Tokens.vocab


In [51]:
train_batches, test_batches, vocab = get_datasets('cuda:0', batch_size=50)

AttributeError: 'Example' object has no attribute 'text'

In [72]:
import torch
from torchtext.datasets import AG_NEWS
train_iter = AG_NEWS(split='train')   # Cf. batches

(3, "Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\\about the economy and the outlook for earnings are expected to\\hang over the stock market next week during the depth of the\\summer doldrums.")


In [74]:
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.utils import get_tokenizer
tokenizer = TweetTokenizer().tokenize
# tokenizer = get_tokenizer('basic_english')

def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

In [75]:
vocab(['here', 'is', 'an', 'example'])

[525, 24, 33, 5982]

In [104]:
import pandas as pd
pd.read_csv('data/comments_positive.csv')[:5]

,id,parent_id,subreddit_id,link_id,text,score,ups,author,controversiality,parent_link_id,parent_text,parent_score,parent_ups,parent_author,parent_controversiality
0,c092j8m,t1_c092gss,t5_2qh2p,t3_8eyy3,This isn't Twitter: try to comment on the arti...,9582,9582,nraustinii,0,t3_8eyy3,Fucking faggot.,-7526,-7526,Glorificus,0
1,c4imcva,t1_c4im948,t5_2qh1i,t3_t0ynr,"Well, it is exactly what it sounds like. It's ...",9531,9531,Lynfect,0,t3_t0ynr,"Elaborate on this cum box, please.",3841,3841,eeeeevil,0
2,c0s4nfi,t1_c0s4lje,t5_2qh1i,t3_cf1n2,"In soviet Russia, bomb disarms you!",8545,8545,CapnScumbone,0,t3_cf1n2,"I don't live in Russia anymore, and I will not...",621,621,shady8x,0
3,c4ini33,t1_c4incln,t5_2qh1i,t3_t0ynr,"""runin for senitur! #YOLO!""",7430,7430,[deleted],0,t3_t0ynr,This just made me realize that future presiden...,4651,4651,drspg99,0
4,c4imgel,t1_c4ima2e,t5_2qh1i,t3_t0ynr,You step motherfucker.,7173,7173,jbg89,0,t3_t0ynr,I have sex with my step mom when my dad isn't ...,4251,4251,audir8,0


In [82]:
from torch.nn.utils.rnn import pad_sequence

a = torch.ones(20)
b = torch.ones(12)
c = torch.ones(15)

##### tens = torch.randn(50,150,2)

In [91]:
50%0

ZeroDivisionError: integer division or modulo by zero